# Phys 605 Homework 1 Problem 1

I will first provide the solution for the functional programming method, and next for the object oriented method. If you wrote your answer as a function, please also look at the solution for the object oriented method.

For series, the resulting equivalent resistor is simply:
$$
R_{eq} = R_{1} + R_{2} + R_{3} + ...
$$

For parallel, the resulting equivalent resistor is found from:
$$
\frac{1}{R_{eq}} = \frac{1}{R_1} + \frac{1}{R_2} + \frac{1}{R_3} + ...
$$
For two resistors, this equation allows you to simplify to:
$$
R_{eq} = \frac{R_1 R_2}{R_1 + R_2}
$$

Note that for the first equation, you need to take special care for the situation where one of the resistors has zero resitance, i.e. it is a wire. In that case the equivalent resistor also has zero resistance. For the second formulation, this exception only needs to be handled if *both* resistors are zero.

## Beginner: Functional Programming

We write functions to compute the answers for us.

### a: Functions for Series and Parallel

In [1]:
def Series(R1,R2):
    # The R1 and R2 values can simply be added for series configuration.
    return(R1+R2)

def Parallel(R1,R2):
    # For parallel:
    Req = R1*R2/(R1 + R2)
    return(Req)

Test the code:

In [2]:
print( Series(10,20))  # Should print 30
print( Parallel(10,20)) # Should print 6.66666
print( Parallel(10,0))  # This should print 0
print( Parallel(0,0))   # This should give 0, but returns an error.

30
6.666666666666667
0.0


ZeroDivisionError: division by zero

### b: Update the code to accept any number of resistors.

The simple way is to accept a list, so you would call the function as: `Series([1,2,3])`, but
you may also want to use `Series(1,2,3)`. The way to accomplish the latter is to use `Series(*Rs)`. 
However, a good design would be to permit *both* types of input. To do that, you need to test what it 
was that you received for argument. The `Rs` variable will always be of *type* 'tuple', but in the second
version (`Series(*Rs)`) the tuple with be of length `len(Rs)` larger than 1, and contain numbers, while in the first version (`Series([1,2,3])`) it will be of length 1, and that first element will be a list or a tuple. We thus need an `if` statement to sort out what we got. The safest would be to test `Rs[0]` for the type. If it is a number, then use the `Rs` as the list, if not, use `Rs[0]` as the list. The safest way to test if `Rs[0]` is a number is `isinstance(Rs[0],numbers.Number)`.

In [4]:
import numbers
def Series(*Rs):
    if isinstance(Rs[0],numbers.Number):
        RR = Rs
    else:
        RR = Rs[0]
    # At this point, RR is the list of numbers either way it was called.
    Result = sum(RR) # Just sum all of them for series.
    return(Result)

def Parallel(*Rs):
    if isinstance(Rs[0],numbers.Number):
        RR = Rs
    else:
        RR = Rs[0]
    # At this point, RR is the list of numbers either way it was called.
    Invs = [1/x for x in RR] # compute the list of inverses.
    Result = 1/sum(Invs) # Just sum all of them and return 1/sum
    return(Result)

In [5]:
print( Series(10,20))  # Should print 30
print( Parallel(10,20)) # Should print 6.66666...
print( Series(10,20,30))  # Should print 60
print( Parallel(10,20,30)) # Should print 5.454545...
print( Parallel(10,0)) # Should print 0, but gives an error!

30
6.666666666666666
60
5.454545454545454


ZeroDivisionError: division by zero

We get a division by zero if one of the values is zero, but zero is a perfectly fine input for a resistor. We need to treat this case special. Note that before the result was OK only if one of the two was zero, but not both. We didn't test that. Here is the improved version:

In [6]:
def Parallel(*Rs):
    if isinstance(Rs[0],numbers.Number):
        RR = Rs
    else:
        RR = Rs[0]
    # At this point, RR is the list of numbers either way it was called.
    if 0 in RR: return(0)
    Invs = [1/x for x in RR] # compute the list of inverses.
    Result = 1/sum(Invs) # Just sum all of them and return 1/sum
    return(Result)

In [7]:
print( Series(10,20))  # Should print 30
print( Parallel(10,20)) # Should print 6.66666...
print( Series(10,20,30))  # Should print 60
print( Parallel(10,20,30)) # Should print 5.454545...
print( Parallel(10,0)) # Should print 0, and now it works!
print( Parallel(0,0))  # This one too! 

30
6.666666666666666
60
5.454545454545454
0
0


### c: Write a function for a resistor divider

In this case, we know the 3 input arguments: Vin, R1 , R2, so no fancy testing is needed. We can simply use the formula from the book, or from lecture:
$$
V_{out} = V_{in} \frac{R_2}{R_1 + R_2}
$$
Note that this is the voltage across $R_2$, which is usually the "bottom" one in the divider. The function is pretty simple.

In [8]:
def Divider(Vin,R1,R2):
    Vout = Vin *R2/(R1+R2)
    return(Vout)

In [9]:
print(Divider(9,20000,10000))  # Should be 3
print(Divider(9,10000,20000))  # Should be 6
print(Divider(9,10000,0))      # Should be 0
print(Divider(9,0,10000))      # Should be 9

3.0
6.0
0.0
9.0


You could now build on these functions for making it easier for you to calculate values when you are working on circuits. 

Note that this solution does not check if your divider goes outside of the permitted power load for the resistors. To see a solution that does this, look at the one below.

## Advanced: Object Oriented Programming

In this style of programming, you create objects that behave in the manner you expect them to. Then you can use these objects to do calculations. It really isn't all that difficult, so "advanced" here is relative. You should be able to understand what this code does.

### a: A basic Resistor class

We create a new Python object for Resistor. The object needs an initializer, `__init__`, which sets the value when create the object. We need to override the operators "+", `__add__`, for series, and "*", `__mul__`, for parallel. We want to be able to print the object, so we need `__str__`.

The simple version is below:

In [10]:
class Resistor():
    
    def __init__(self,R=0):   # The required initializer.
        self._resistance = R
    
    def __add__(self,other):
        return( Resistor(self._resistance + other._resistance))
    
    def __mul__(self,other):
        if self._resistance == 0 or other._resistance == 0:
            return(Resistor(0))
        else:
            return( Resistor(self._resistance*other._resistance/(self._resistance + other._resistance )))
               
    def __str__(self):
        return( str(self._resistance)+"Ω")

In [11]:
R1= Resistor(10.)
R2= Resistor(20.)
R3= Resistor(30.)
print(R1+R2)
print(R1*R2)
print(R1+R2+R3)
print(R1*R2*R3)
print(Resistor(0)*R1)
print(Resistor(0)*Resistor(0))

30.0Ω
6.666666666666667Ω
60.0Ω
5.454545454545455Ω
0Ω
0Ω


So, it works, but I would like a bit more elegance, and also I can forsee the need to want to change the value of a resistor, which in this implementation is not really allowed (the `_resistance` is marked as "private").
We can do this by turning the resistance into a *property*, which I want to call R.
See more about [properties here.](https://www.datacamp.com/community/tutorials/property-getters-setters). Another nice explantion [about properties here.](https://www.python-course.eu/python3_properties.php)

In [12]:
class Resistor():
       
    def __init__(self,R=0):   # The required initializer.
        self.R = R
    
    def __add__(self,other):
        return( Resistor(self.R + other.R))
    
    def __mul__(self,other):
        if self.R == 0 or other.R == 0:
            return(Resistor(0))
        else:
            return( Resistor(self.R*other.R/(self.R + other.R )))
               
    def __str__(self):
        return( str(self.R)+"Ω")
    
    @property 
    def R(self):
        return(self._resistance)
    
    @R.setter
    def R(self,Rnew):
        if Rnew>= 0:
            self._resistance=Rnew
        else:
            self._resistance=0

In [13]:
R1= Resistor(10.)
R2= Resistor(20.)
R3= Resistor(30.)
print(R1+R2)
print(R1*R2)
print(R1+R2+R3)
print(R1*R2*R3)
print(Resistor(0)*R1)
print(Resistor(0)*Resistor(0))

30.0Ω
6.666666666666667Ω
60.0Ω
5.454545454545455Ω
0Ω
0Ω


OK, that works better, but I am *still* not quite happy with it yet. We also want to be able to do $R\cdot 2$ or $2\cdot R$, i.e. multiply the value of the resistor by a number, which is useful for formulas. To do this we need to check the type, and we need to implement `__rmul__` for that second situation.

We will also encounter situations where you *divide* resistor values. The result should be a number, not a resistor. We implement this with `__truediv__`, which in Python3 is used for the "/" operation. (Note that in Python2 you would use `__div__`.) We should also implement the `__floordiv__` operator, which corresponds to "//". Again, we want to be careful to treat dividing by a number different from dividing by a Resistor. We would also want to be able to divide a number by a Resistor, with requires implementing `__rtruediv__` and `__rfloordiv__`, similar to `__rmult__`. If (when?) you write a complete electronics package, you would want to do things like divide a voltage (V) by a resistor, which you can do when you have a class for voltage, and one for current. In such a more complete package you can then also keep proper track of the units.

With divide, we again need to worry about divide by zero. In this case, the result shold be infinity, which you can get from 

The better class is then:

In [14]:
import numbers
import math
class Resistor():
    
    def __init__(self,R=0):   # The required initializer.
        self.R = R
    
    def __add__(self,other):
        return( Resistor(self.R + other.R))
    
    def __mul__(self,other):
        if isinstance(other,numbers.Number):
            return(Resistor(self.R * other))
        if self.R == 0 or other.R == 0:
            return(Resistor(0))
        else:
            return( Resistor(self.R*other.R/(self.R + other.R )))
        
    def __rmul__(self,other):  # Note, __rmul__ is called for 2*R situation.
        return(Resistor(self.R * other))
        
    def __truediv__(self,other):
        if isinstance(other,numbers.Number):
            return(Resistor(self.R /other))   # Returns a Resistor.
        if other.R == 0:
            return(Resistor(math.inf)) 
        else:
            return( self.R/other.R) # Returns a number!

    def __floordiv__(self,other):
        if isinstance(other,numbers.Number):
            return(Resistor(self.R //other))   # Returns a Resistor.
        if other._resistance == 0:
            return(Resistor(math.inf)) 
        else:
            return( self.R//other.R) # Returns a number!
        
    def __rtruediv__(self,other):
        return( other/self.R) # Returns a number!

    def __rfloordiv__(self,other):
        return( other//self.R) # Returns a number!

        
    def __str__(self):
        return( str(self.R)+"Ω")
       
    @property 
    def R(self):
        return(self._resistance)
    
    @R.setter
    def R(self,Rnew):
        if Rnew>= 0:
            self._resistance=Rnew
        else:
            self._resistance=0

In [15]:
print(Resistor(20)*2)
print(2*Resistor(20))
print(Resistor(20)/2)   # True division
print(Resistor(20)//2)  # Integer division
print(200/Resistor(20))   # True division
print(200//Resistor(20))  # Integer division
print(Resistor(200)/Resistor(100))
print(Resistor(200)//Resistor(100))
print(Resistor(200)/Resistor(0))
print(Resistor(0)/Resistor(200))

40Ω
40Ω
10.0Ω
10Ω
10.0
10
2.0
2
infΩ
0.0


### b: Improved Resistor class

So, our Resistor was nice and useful, but a physical resistor also has a power limit. We want to add this to our resistor class. We will write it so that if the power is not specified, it will use 1/8 Watt power value.

We now have to be really careful with the power limits. They do not simply add. So how do we calculate the new power limit?

For series, we know the same current will flow through both resistors. The power dissipated in each resistor is given by $P = R*I^2$, so $I_{max} = \sqrt{P_{max}/R}$. We thus need to calculate the $I_{max}$ for each resistor, and choose the *smaller* of the two. The total power of the combined resistors in series is then $P_{max} = (R_1 + R_2)*I_{max}^2$.   

For parallel, we know that the same voltage is across both resistors. The power dissipated in each resistor is then 
given by: $P = V^2/R$, so $V_{max} = \sqrt{P_{max}*R}$. We now, like before, need to choose the smallest $V_{max}$. The total maximum power of the combined resistors in parallel is then: $P_{max} = V_{max}^2(1/R_1 + 1/R_2)$.

We should add two methods (functions) that return the resistance and the power rating.

We add some additional methods (functions) that allow us to quickly compute and check if the power use in the resistor is OK or not. When we know the current through the Resistor, we use `PowerI(I)` and `checkPowerI(I)` and when we know the voltage across the resistor, `PowerV(V)` and `checkPowerV(V)`.

In [22]:
import numbers
import math
class Resistor():
        
    def __init__(self,R=0,P=1./8.):   # The required initializer.
        self.R = R
        self.Pmax = P
    
    def __add__(self,other):        # Implement the "+" operation.
        Value = self.R + other.R
        I2_max = min(self.Pmax/self.R,other.Pmax/other.R) # I_max squared.
        Power = I2_max*(self.R + other.R)
        return( Resistor(Value,Power))
    
    def __mul__(self,other):                           # Implement the "*" operation. We use this for parallel.
        if isinstance(other,numbers.Number):           # But if other is a number, then multiply.
            return(Resistor(self.R * other,self.Pmax)) # Power rating not modified for numeric multiply.
        if self.R == 0 or other.R == 0:
            return(Resistor(0,math.inf))               # A zero ohm resistor has infinite power :-)
        else:
            Value = self.R*other.R/(self.R + other.R)
            V2_max = min(self.Pmax*self.R,other.Pmax*other.R) # V_max squared.
            Power = V2_max*(1/self.R + 1/other.R)
            return( Resistor(Value,Power))             # Return the parallel equivalent resistor
 
    def __or__(self,other):                            # An alternate for "*" could be "|", but no one uses that.
        if isinstance(other,numbers.Number):
            return(Resistor(self.R * other,self.Pmax)) # Power rating not modified.
        if self.R == 0 or other.R == 0:
            return(Resistor(0,math.inf))               # A zero ohm resistor has infinite power :-)
        else:
            Value = self.R*other.R/(self.R + other.R)
            V2_max = min(self.Pmax*self.R,other.Pmax*other.R) # V_max squared.
            Power = V2_max*(1/self.R + 1/other.R)
            return( Resistor(Value,Power))           # Return the parallel equivalent resistor


    def __rmul__(self,other):                        # Note, __rmul__ is called for 2*R situation.
        return(Resistor(self.R * other,self.Pmax))      # Then other is always a number, so no need to test.
        
    def __truediv__(self,other):                     # Implement division with "/".
        if isinstance(other,numbers.Number):         # If other is a number, then divide the value
            return(Resistor(self.R /other,self.Pmax))   # and return a Resistor.
        if other.R == 0:                             # If other is zero, you get an open connection = infinity.
            return(Resistor(math.inf,self.Pmax)) 
        else:
            return( self.R/other.R)                  # Divide resistors: returns a number! Power info is lost.

    def __floordiv__(self,other):                     # Implement integer division "//"
        if isinstance(other,numbers.Number):          # If other is a number, then divide the value
            return(Resistor(self.R //other,self.Pmax))   # Returns a Resistor.
        if other.R == 0:                              # If other is zero, you get an open connection = infinity.
            return(Resistor(math.inf,self.Pmax)) 
        else:
            return( self.R//other.R)                  # Integer Divide resistors: returns a number! Power info is lost.
     
    def __rtruediv__(self,other):                     # This is for  2/R so a number on the left.
        return( other/self.R) # Returns a number!     # Other is always a number, return a number.

    def __rfloordiv__(self,other):                    # This is for 2//R so number on the left.
        return( other//self.R) # Returns a number!    # Other is always a number, return a number.

    def PowerI(self,I):                               # Calculate the power used for current I.
        return(I*I*self.R)
    
    def checkPowerI(self,I):                          # Check if power rating is sufficient for current I
        return( self.PowerI(I)<self.Pmax)

    def PowerV(self,V):                               # Calculate the power used if connected to voltage V
        return(V*V/self.R)                            
    
    def checkPowerV(self,V):                          # Check the power rating if connected to voltage V.
        return( self.PowerV(V)<self.Pmax)
    
    def __str__(self):                                # Print a sensible string with the resistance and the power.
        return( str(self.R)+"Ω," + str(self.Pmax)+"W")
    
    @property                                         # This makes "R" a property of the resistor.
    def R(self):
        return(self._resistance)
    
    @R.setter                                         # This allows you to set the property R
    def R(self,Rnew):
        if Rnew>= 0:                                  # We test to make sure the value makes sense.
            self._resistance=Rnew
        else:
            print("Error, you cannot have a negative resistance. (At least not here.)")
            self._resistance=0
            
    @property                                        # This allows you to set the property Pmax
    def Pmax(self):
        return(self._power)
    
    @Pmax.setter
    def Pmax(self,P):                                # This allows you to set Pmax.
        if P>= 0:
            self._power=P
        else:
            print("Error - we cannot have negative power.")
            self._power=0
            

In [23]:
R1 = Resistor(1000,1/8)
R2 = Resistor(2000,1/8)
print(R1+R1+R1+R1)
print(R1 | R2)

4000Ω,0.5W
666.6666666666666Ω,0.1875W


### c: Voltage divider function, using the Resistor class.

So now we can write a voltage divider function that can use the resistor class. The problem did not specify it, but because we now have resistors with a power rating, we can check to make sure our voltage divider does not fry either of the resistors.

Note how the use of our Resistor class makes it easier to 



In [24]:
def Divider(Vin,R1,R2):
    if isinstance(R1,Resistor) and isinstance(R2,Resistor):  # Both are R, so we can check power.
        Vout = Vin *R2/(R1+R2)
        I = Vin/(R1+R2)
        if not R1.checkPowerI(I) or not R2.checkPowerI(I):
            print("Error, power rating exceeded!")
            return(0)
        else:
            return(Vout)
    else:
        Vout = Vin *R2/(R1+R2)  # Assume both are numbers then. You get an error for mixed input!

In [25]:
print(Divider(10,Resistor(100,1),Resistor(100,1)),"V")
print(Divider(10,Resistor(10,1),Resistor(20,1)),"V")

5.0 V
Error, power rating exceeded!
0 V


In [35]:
R1 = Resistor(10)
R2 = Resistor(20)
R1.Pmax = 2.25
R2.Pmax = 2.25
print(R1+R2,(R1+R2).PowerV(10),"W")
print(Divider(10,R1,R2),"V")

30Ω,3.375W 3.3333333333333335 W
6.666666666666667 V
